# Portfolio Performance vs. Benchmark

### Step 1: Import Libraries

In [1]:
# Data manipulation libraries
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from datetime import datetime, timedelta
from pandas.tseries.offsets import BDay

# Visualization libraries
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import seaborn as sns
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import Image, display

# System libraries
import os
import sys
import logging
import warnings

warnings.filterwarnings("ignore", category=UserWarning)  # Font warnings
warnings.filterwarnings("ignore", category=FutureWarning, message=".*inplace.*")  # Pandas warnings
logging.getLogger('matplotlib.font_manager').disabled = True

In [2]:
# UDFs
current_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'py')) 
sys.path.append(current_dir)
from quantstats_fix import *
from utils import load_and_filter_data, export_to_excel
qs.extend_pandas()

               QuantStats Compatibility Tool                

Part 1: Directly patching QuantStats package files
------------------------------------------------------------
Found QuantStats utils file at: /opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/quantstats/__init__.py
Creating backup at: /opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/quantstats/__init__.py.bak
Successfully fixed indentation in QuantStats __init__.py file
✓ QuantStats utils file patched successfully

Part 2: Fixing resampling issues
------------------------------------------------------------
Found 1 potential QuantStats installation(s)
Checking /opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/quantstats/_plotting/core.py
✓ Found 'plot_timeseries' function in /opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/quantstats/_plotting/core.py
✓ No 'sum(axis=0)' calls found - may already be fixed
Examining /opt/hostedtoolcache/Python/3.12.3/x

### Step 2: Define Parameters 

#### Dates

In [3]:
# Define the date range
end_date = (datetime.today() - BDay(1)).to_pydatetime()  # Subtract 1 business day
# end_date = pd.to_datetime('2025-04-26')  # Report date
start_date = end_date - timedelta(days=5*365)

# Convert datetime objects to Unix timestamps (seconds since Jan 1, 1970)
start_timestamp = int(start_date.timestamp())
end_timestamp = int(end_date.timestamp())

# Print the date range
days_difference = (end_date - start_date).days
print(f"Date Range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
print(f"Time span: {days_difference} days ({days_difference/365:.2f} years)")

Date Range: 2020-06-07 to 2025-06-06
Time span: 1825 days (5.00 years)


#### Report File

In [4]:
output_file = f'portfolio-{datetime.date(end_date)}.xlsx'

# benchmark = 'ADME'
benchmark = pd.read_excel(output_file, sheet_name="benchmark")['Benchmark'].values[0]
print(benchmark)

DLN


#### Risk-free rate (T-bill, %)

In [5]:
# Load and process data
risk_free_df = pd.read_excel(output_file, sheet_name="daily_quotes", index_col=0)['^IRX']
# risk_free_rate  = 0.0433                              # 3‑month T‑Bill
risk_free_rate = risk_free_df.iloc[-1] / 100 

# Display result
print("Risk-Free Rate:", risk_free_rate, "-- 13 WEEK TREASURY BILL (^IRX)")

Risk-Free Rate: nan -- 13 WEEK TREASURY BILL (^IRX)


### Step 3: Read Portfolio Data (Excel)

In [6]:
portfolio_df = pd.read_excel(output_file, sheet_name="equity")

# Convert percentage strings to float values
portfolio_df['Weight'] = portfolio_df['Weight'].replace('%', '', regex=True).astype(float)
portfolio_tickers = portfolio_df["Ticker"].tolist()

print(portfolio_tickers)
display(portfolio_df)

['PGR', 'DE', 'TMUS', 'XOM', 'GILD', 'AMAT', 'HD', 'PG']


,Ticker,Date,Name,Sector,Industry,Country,Website,Market Cap,Enterprise Value,Float Shares,...,52W High,52W Low,50 Day Avg,200 Day Avg,Short Ratio,Short % of Float,Weight,Expected Return,Standard Deviation,Sharpe Ratio
0,PGR,2025-06-09,The Progressive Corporation,Financial Services,Insurance - Property & Casualty,United States,https://www.progressive.com,163744088064,167847985152,584037029,...,292.99,201.34,277.12,260.47,2.64,0.01,0.400000,0.266580,0.250331,1.064911
1,DE,2025-06-09,Deere & Company,Industrials,Farm & Heavy Construction Machinery,United States,https://www.deere.com/en,140827328512,201164357632,270233944,...,533.78,340.20,480.48,445.10,3.64,0.02,0.207366,0.212028,0.293837,0.721583
2,TMUS,2025-06-09,"T-Mobile US, Inc.",Communication Services,Telecom Services,United States,https://www.t-mobile.com,279161733120,388849532928,452736028,...,276.49,171.18,249.73,234.17,3.11,0.03,0.142634,0.157460,0.241697,0.651479
3,XOM,2025-06-09,Exxon Mobil Corporation,Energy,Oil & Gas Integrated,United States,https://corporate.exxonmobil.com,449366163456,476967043072,4300373098,...,126.34,97.80,106.58,112.13,3.05,0.01,0.050000,0.133824,0.298778,0.447904
4,GILD,2025-06-09,"Gilead Sciences, Inc.",Healthcare,Drug Manufacturers - General,United States,https://www.gilead.com,139892359168,156049276928,1241366627,...,119.96,62.69,105.90,96.25,2.68,0.02,0.050000,0.090207,0.231579,0.389529
5,AMAT,2025-06-09,"Applied Materials, Inc.",Technology,Semiconductor Equipment & Materials,United States,https://www.appliedmaterials.com,133808521216,133731549184,799392549,...,255.89,123.74,152.45,172.28,2.58,0.02,0.050000,0.124842,0.431222,0.289508
6,HD,2025-06-09,"The Home Depot, Inc.",Consumer Cyclical,Home Improvement Retail,United States,https://www.homedepot.com,365466877952,426049896448,993306252,...,439.37,326.31,362.70,387.72,3.19,0.01,0.050000,0.069800,0.237976,0.293306
7,PG,2025-06-09,The Procter & Gamble Company,Consumer Defensive,Household & Personal Products,United States,https://www.pginvestor.com,384551452672,410630782976,2341001776,...,180.43,156.58,164.42,168.41,2.31,0.01,0.050000,0.075310,0.172571,0.436400


### Step 4: Download Returns

In [7]:
# stock_returns = qs.utils.download_returns(ticker=portfolio_tickers, period="5y").dropna()
stock_quotes = load_and_filter_data('../data/datasets/daily_stock_quotes.csv', portfolio_tickers, start_date, end_date)
stock_returns = np.log(stock_quotes / stock_quotes.shift(1)).dropna()

benchmark_quotes = load_and_filter_data('../data/datasets/daily_benchmark_quotes.csv', benchmark, start_date, end_date)
benchmark_returns = np.log(benchmark_quotes / benchmark_quotes.shift(1)).dropna()

# Display summary statistics for all assets
print("\nSharpe Ratios for individual assets:")
sharpes = {col: qs.stats.sharpe(stock_returns[col]) for col in stock_returns.columns}
for ticker, sharpe in sharpes.items():
    print(f"{ticker}: {sharpe:.4f}")

display(stock_returns.head())

Found 8 of 8 tickers in ../data/datasets/daily_stock_quotes.csv
Missing tickers: []
Found 1 of 1 tickers in ../data/datasets/daily_benchmark_quotes.csv
Missing tickers: []

Sharpe Ratios for individual assets:
AMAT: 0.4891
DE: 0.8020
GILD: 0.4883
HD: 0.4030
PG: 0.5074
PGR: 1.0707
TMUS: 0.7264
XOM: 0.5698


,AMAT,DE,GILD,HD,PG,PGR,TMUS,XOM
Date,,,,,,,,
2020-06-09,-0.001742,-0.023965,0.007918,-0.000044,-0.006003,-0.005984,-0.012213,-0.022488
2020-06-10,0.001046,-0.023424,-0.006647,-0.009034,0.007522,-0.009181,0.024859,-0.055188
2020-06-11,-0.077114,-0.063517,-0.047640,-0.060660,-0.025264,-0.055912,-0.047409,-0.092341
2020-06-12,0.020304,0.025355,0.004983,0.012375,-0.005463,0.008986,0.013990,0.021240
2020-06-15,0.008812,0.003236,0.010385,-0.004541,0.009153,0.034459,0.022924,-0.000798


### Step 5: Plot Return Comparisons

In [8]:
# Calculate portfolio returns using weights from Excel file
portfolio_weights = portfolio_df.set_index('Ticker')['Weight'].to_dict()
weighted_returns = pd.DataFrame()

print("\nPortfolio Weights:")
for ticker in portfolio_tickers:
    weight = portfolio_weights.get(ticker, 0)
    print(f"{ticker}: {weight:.2%}")
    if ticker in stock_returns.columns:
        weighted_returns[ticker] = stock_returns[ticker] * weight

# Sum across all weighted returns to get the portfolio return
portfolio_return = weighted_returns.sum(axis=1)

# Create equal-weight portfolio for comparison
equal_weight = 1/len([t for t in portfolio_tickers if t in stock_returns.columns])
equal_weighted_returns = pd.DataFrame()

for ticker in portfolio_tickers:
    if ticker in stock_returns.columns:
        equal_weighted_returns[ticker] = stock_returns[ticker] * equal_weight
        
equal_weight_return = equal_weighted_returns.sum(axis=1)

print("\nPortfolio Performance Summary:")
print(f"Sharpe Ratio (Weighted Portfolio): {qs.stats.sharpe(portfolio_return):.4f}")
print(f"Sharpe Ratio (Equal-Weight): {qs.stats.sharpe(equal_weight_return):.4f}")

plt.figure(figsize=(12, 6))
(1 + portfolio_return).cumprod().plot(label='Weighted Portfolio')
(1 + equal_weight_return).cumprod().plot(label='Equal Weight')
(1 + benchmark_returns).cumprod().plot(label=benchmark)
plt.legend()
plt.title('Performance Comparison')
plt.ylabel('Cumulative Return')
plt.grid(True)
plt.show()


Portfolio Weights:
PGR: 40.00%
DE: 20.74%
TMUS: 14.26%
XOM: 5.00%
GILD: 5.00%
AMAT: 5.00%
HD: 5.00%
PG: 5.00%

Portfolio Performance Summary:
Sharpe Ratio (Weighted Portfolio): 1.2530
Sharpe Ratio (Equal-Weight): 1.0680


### Step 6: Generate Reports

#### Portfolio (Weighted) vs Benchmark

In [9]:
portfolio_return = portfolio_return.resample('D').sum()  # Example of valid aggregation
portfolio_return.name = "Weighted Portfolio"

# Generate Report
qs.reports.html(
    portfolio_return,
    benchmark_returns,
    rf=risk_free_rate,
    figsize=(8, 5),
    # output=f'portfolio_vs_{benchmark}.html',
    output=f'portfolio_vs_benchmark-{datetime.date(end_date)}.html',
    title=f'Portfolio vs {benchmark} (Benchmark)',
    benchmark_title=f'{benchmark}',
    download_filename=f'portfolio_vs_{benchmark}.html'
)

qs.reports.full(
    portfolio_return, 
    benchmark_returns,
    rf=risk_free_rate, 
    figsize=(8, 5), 
    title=f'Portfolio vs {benchmark}',
    benchmark_title=f'{benchmark}') 

/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, p

Added download button and removed QuantStats attribution from portfolio_vs_benchmark-2025-06-06.html


/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, p

                           DLN         Strategy
-------------------------  ----------  ----------
Start Period               2020-06-10  2020-06-10
End Period                 2025-06-05  2025-06-05
Risk-Free Rate             -           -
Time in Market             69.0%       69.0%

Cumulative Return          72.59%      173.78%
CAGR﹪                     7.84%       14.96%

Sharpe                     0.69        1.05
Prob. Sharpe Ratio         -           -
Smart Sharpe               0.69        1.04
Sortino                    0.97        1.51
Smart Sortino              0.96        1.5
Sortino/√2                 0.68        1.07
Smart Sortino/√2           0.68        1.06
Omega                      1.24        1.24

Max Drawdown               -17.03%     -12.4%
Longest DD Days            601         234
Volatility (ann.)          11.99%      14.25%
R^2                        0.39        0.39
Information Ratio          0.04        0.04
Calmar                     0.46        1.21
Skew  

None

,Start,Valley,End,Days,Max Drawdown,99% Max Drawdown
1,2022-04-11,2022-06-16,2022-08-16,128,-12.395596,-12.109405
2,2021-05-17,2021-10-12,2022-01-05,234,-11.404492,-10.841998
3,2025-03-18,2025-04-08,2025-05-18,62,-10.702499,-10.470111
4,2024-12-02,2025-01-10,2025-02-12,73,-10.687040,-9.957934
5,2022-08-26,2022-09-30,2022-10-27,63,-10.601741,-9.993238


/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/quantstats/_plotting/wrappers.py:555: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  .resample("A")
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/quantstats/_plotting/wrappers.py:557: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  .resample("A")
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/quantstats/_plotting/wrappers.py:565: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  returns = returns.resample("A").apply(_stats.comp)
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/quantstats/_plotting/wrappers.py:568: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  returns = returns.resample("A").last()
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.1

#### Portfolio (Weighted) vs Portolio (Equally-Weighted)

In [10]:
# Ensure equal_weight_return and portfolio_return are properly aggregated if resampled
equal_weight_return = equal_weight_return.resample('D').sum() if equal_weight_return.index.freq is None else equal_weight_return
portfolio_return = portfolio_return.resample('D').sum() if portfolio_return.index.freq is None else portfolio_return

# Set the name for the equal weight portfolio
equal_weight_return.name = "Equal Weight Portfolio"

# Generate Report
qs.reports.html(
    portfolio_return,
    equal_weight_return,
    rf=risk_free_rate,
    figsize=(8, 5),
    output=f'portfolio_vs_equal_weight-{datetime.date(end_date)}.html',
    title='Portfolio (Weighted) vs Portolio (Equally-Weighted)',
    benchmark_title="Equal Weight Portfolio",
    download_filename="portfolio_vs_equal_weight.html" 
)

qs.reports.full(
    portfolio_return, 
    equal_weight_return,
    rf=risk_free_rate, 
    figsize=(8, 5), 
    title='Portfolio vs Equal Weight',
    benchmark_title="Equal Weight Portfolio",
    ) 

/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, p

Added download button and removed QuantStats attribution from portfolio_vs_equal_weight-2025-06-06.html


/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, p

                           Equal Weight Portfolio    Strategy
-------------------------  ------------------------  ----------
Start Period               2020-06-09                2020-06-09
End Period                 2025-06-05                2025-06-05
Risk-Free Rate             -                         -
Time in Market             69.0%                     69.0%

Cumulative Return          118.5%                    170.98%
CAGR﹪                     11.42%                    14.78%

Sharpe                     0.89                      1.04
Prob. Sharpe Ratio         -                         -
Smart Sharpe               0.88                      1.03
Sortino                    1.28                      1.49
Smart Sortino              1.27                      1.49
Sortino/√2                 0.9                       1.06
Smart Sortino/√2           0.9                       1.05
Omega                      1.24                      1.24

Max Drawdown               -13.49%              

None

,Start,Valley,End,Days,Max Drawdown,99% Max Drawdown
1,2022-04-11,2022-06-16,2022-08-16,128,-12.395596,-12.109405
2,2021-05-17,2021-10-12,2022-01-05,234,-11.404492,-10.841998
3,2025-03-18,2025-04-08,2025-05-18,62,-10.702499,-10.470111
4,2024-12-02,2025-01-10,2025-02-12,73,-10.687040,-9.957934
5,2022-08-26,2022-09-30,2022-10-27,63,-10.601741,-9.993238


/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/quantstats/_plotting/wrappers.py:555: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  .resample("A")
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/quantstats/_plotting/wrappers.py:557: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  .resample("A")
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/quantstats/_plotting/wrappers.py:565: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  returns = returns.resample("A").apply(_stats.comp)
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/quantstats/_plotting/wrappers.py:568: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  returns = returns.resample("A").last()
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.1